# Get data

In [90]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../db/pucker.sqlite3')

df = pd.read_sql_query("SELECT * FROM states", conn)
df = df.loc[:, 'total_players':'reward'] # Remove ID

df_flop = df[df.turn_rank.isnull() & df.river_rank.isnull()]
df_flop = df_flop.drop(['turn_rank', 'turn_suit', 'river_rank', 'river_suit'], axis=1)

Xflop = df_flop.loc[:, 'total_players':'decision_raise']
yflop = df_flop.loc[:, 'reward']
yflop = pd.DataFrame([1 if item > 0 else 0 for item in yflop])

df_turn = df[df.turn_rank.notnull() & df.river_rank.isnull()]
df_turn = df_turn.drop(['river_rank', 'river_suit'], axis=1)

Xturn = df_turn.loc[:, 'total_players':'decision_raise']
yturn = df_turn.loc[:, 'reward']

df_river = df[df.turn_rank.notnull() & df.river_rank.notnull()]

Xriver = df_river.loc[:, 'total_players':'decision_raise']
yriver = df_river.loc[:, 'reward']
    

   # Encode and scale

In [91]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer

def diff(first, second):
    first = list(first)
    second = list(second)
    return [item for item in first if item not in second]

def process(data):
    encode_columns = [item for item in data.columns if 'suit' in item]
    scale_columns = diff(data.columns, encode_columns)

    column_transformer = make_column_transformer(
        (StandardScaler(), scale_columns),
        (OneHotEncoder(categories='auto'), encode_columns))

    return column_transformer.fit_transform(data)




# Build model

In [92]:
from keras.models import Sequential
from keras.layers import Dense

def neural_network(nfeatures):
    model = Sequential()

    model.add(Dense(input_dim=nfeatures, units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='tanh', kernel_initializer='glorot_normal'))
    model.add(Dense(units=1000, activation='tanh', kernel_initializer='glorot_normal'))
    model.add(Dense(units=1, activation='linear', kernel_initializer='random_normal'))

    model.compile(optimizer='adam', loss='mae', metrics=['mae'])

    return model


def flop_neural_network(nfeatures):
    model = Sequential()

    model.add(Dense(input_dim=nfeatures, units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='relu', kernel_initializer='random_normal'))
    model.add(Dense(units=1000, activation='tanh', kernel_initializer='glorot_normal'))
    model.add(Dense(units=1000, activation='tanh', kernel_initializer='glorot_normal'))
    model.add(Dense(units=1, activation='sigmoid', kernel_initializer='random_normal'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'mae'])

    return model

   # Run model

In [93]:
from sklearn.model_selection import train_test_split

X = process(Xflop)
y = yflop.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)

nfeatures = X.shape[1]
model = flop_neural_network(nfeatures)
model.fit(Xtrain, ytrain, batch_size=10000, epochs=50) # 20 for flop 100 for river
ypred = model.predict(Xtest)
    

/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Epoch 1/50
211956/211956 [==============================] - 17s 81us/step - loss: 0.7161 - acc: 0.7440 - mean_absolute_error: 0.3016
Epoch 2/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3416 - acc: 0.7983 - mean_absolute_error: 0.2401
Epoch 3/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3361 - acc: 0.8051 - mean_absolute_error: 0.2362
Epoch 4/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3326 - acc: 0.8121 - mean_absolute_error: 0.2328
Epoch 5/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3290 - acc: 0.8177 - mean_absolute_error: 0.2298
Epoch 6/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3250 - acc: 0.8209 - mean_absolute_error: 0.2257
Epoch 7/50
211956/211956 [==============================] - 15s 73us/step - loss: 0.3234 - acc: 0.8235 - mean_absolute_error: 0.2246
Epoch 8/50
211956/211956 [==============================] - 15s 73us/

# Metrics

In [97]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error

ytest_classification = [1 if item > 0.5 else 0 for item in ytest]
ypred_classification = [1 if item > 0.5 else 0 for item in ypred]

cm = confusion_matrix(ytest_classification, ypred_classification)
print('\nConfusion matrix:')
print(cm)
print("\nLoss classified as loss", cm[0][0])
print("Wins classified as wins", cm[1][1])
print("Wins classified as loss", cm[1][0])
print("Loss classified as wins", cm[0][1])
print('\nAccuracy:\t', accuracy_score(ytest_classification, ypred_classification))
print('Precision:\t', precision_score(ytest_classification, ypred_classification))
print('Recall: \t', recall_score(ytest_classification, ypred_classification))
print('F1 score:\t', f1_score(ytest_classification, ypred_classification))
print('Mean abs error: ', mean_absolute_error(ytest, ypred))


Confusion matrix:
[[36208  5900]
 [ 5439  5442]]

Loss classified as loss 36208
Wins classified as wins 5442
Wins classified as loss 5439
Loss classified as wins 5900

Accuracy:	 0.7860121912094963
Precision:	 0.4798095573972844
Recall: 	 0.5001378549765646
F1 score:	 0.48976285829995947
Mean abs error:  0.21494969869031577


# Save the model in h5 format 

In [ ]:
model.save("pucker.h5")